In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
from sklearn_som.som import SOM

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir1='/Volumes/LaCie/'
dir3='/Volumes/My Passport/day/'

In [ ]:
ds=xr.open_dataset(dir3+'2000/3B-DAY.MS.MRG.3IMERG.20000601-S000000-E235959.V07.nc4')
print(ds)
print(ds['precipitation'].attrs["long_name"])

In [ ]:
yrs=np.arange(2000,2006,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    dir_year=dir3+str(year)+'/'
    days_year=365
    if year%4==0:
        days_year=366
    
    if year==2000:
        date_range=[dt.datetime(2000,6,1,0)+dt.timedelta(days=x) for x in range(214)]

    elif year==2023:
        date_range=[dt.datetime(2023,1,1,0)+dt.timedelta(days=x) for x in range(120)]

    else:
        date_range=[dt.datetime(year,1,1,0)+dt.timedelta(days=x) for x in range(days_year)]
        
    for j in range(len(date_range)):
        date=date_range[j]
        year=date.year
        month=date.month
        day=date.day
        
        if month<10:
            month_str='0'+str(month)
        else:
            month_str=str(month)
        if day<10:
            day_str='0'+str(day)
        else:
            day_str=str(day)
            
        date_str=str(year)+month_str+day_str
        print(date_str)
    
        ds=xr.open_dataset(dir_year+'3B-DAY.MS.MRG.3IMERG.'+date_str+'-S000000-E235959.V07.nc4')
        precip_sel=ds['precipitation']
        precip_sel=precip_sel.transpose('time','lat','lon')
        ds.close()
        
        lons_west=[x for x in precip_sel.lon.values if x<-90]
        lons_east=[x for x in precip_sel.lon.values if x>=90]
        precip_west=precip_sel.sel(lon=lons_west)
        precip_east=precip_sel.sel(lon=lons_east)
        precip_west['lon']=[x+360. for x in lons_west]
        precip_east['lon']=[x-360. for x in lons_east]
        precip_sel=xr.concat([precip_east,precip_sel,precip_west],dim='lon')
        precip_west.close()
        precip_east.close()
    
        precip_sel=precip_sel.interp(lat=np.arange(-90,90,0.25),lon=np.arange(-270,270,0.25))
        lons_center=[x for x in precip_sel.lon.values if -180<=x<180]
        precip_sel=precip_sel.sel(lon=lons_center)
        

        if j==0:
            precip_concat=precip_sel
        else:
            precip_concat=xr.concat([precip_concat,precip_sel],dim='time')
            
        precip_sel.close()
        
    dk=xr.Dataset()
    dk['precip']=(('time','lat','lon'),precip_concat.values)
    dk.coords['time']=precip_concat.time.values
    dk.coords['lat']=precip_concat.lat.values
    dk.coords['lon']=precip_concat.lon.values
    
    dk['time'].attrs["description"]='Datetime'
    dk['lat'].attrs["units"]='degrees_north'
    dk['lon'].attrs["units"]='degrees_east'
    dk['precip'].attrs["units"]='mm*day-1'
    dk['precip'].attrs["description"]='Daily mean precipitation rate (combined microwave-IR) estimate. Formerly precipitationCal.'
    
    try:
        os.remove(dir3+'imerg_combined_deres_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir3+'imerg_combined_deres_'+str(year)+'.nc',mode='w',format='NETCDF4')
    dk.close()